In [134]:
%reset -sf

In [135]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

In [136]:
srr = """
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
""".strip()

In [137]:
with open("/Users/hkmac/Downloads/input.txt") as f:
    srr = f.read().strip()

In [138]:
inp = srr.split("\n")

In [139]:
# [[x for x in row] for row in inp]

In [140]:
row = inp[0]

In [141]:
for row in inp

SyntaxError: invalid syntax (<ipython-input-141-14855250c2c0>, line 1)

In [142]:
row

'{<{[<((<[<({<{(){}}(()())>[<<><>>((){})]}(<<{}{}>(<><>)>))(<[(<>{}){()()}]{{{}[]}(()())}>['

In [143]:
res = 0
arr = []

for row in inp:
#     print(row)
    cn = []
    for c in row:
#         print(cn)

        if c == "(" or c == "[" or c == "{" or c == "<":
            cn.append(c)
            continue
        
        if c == ")":
            if not cn or cn[-1] != "(":
                res += 3
#                 print(1)
                break

        if c == "]":
            if not cn or cn[-1] != "[":
                res += 57
#                 print(2)
                break

        if c == "}":
            if not cn or cn[-1] != "{":
                res += 1197
#                 print(3)
                break
                
        if c == ">":
            if not cn or cn[-1] != "<":
                res += 25137
#                 print(4)
                break
                
        cn.pop()
    
    else:
        print(cn)
        
        score = 0
        for x in cn[::-1]:
            score = score*5
            if x == "(":
                score += 1
            if x == "[":
                score += 2              
            if x == "{":
                score += 3
            if x == "<":
                score += 4
        arr.append(score)

['{', '<', '{', '[', '<', '(', '(', '<', '[', '<', '(', '[']
['[', '<', '{', '{', '{', '[', '[', '(', '<', '(', '{', '[', '(', '[']
['{', '<', '[', '<', '[', '[', '<', '<', '(', '<', '{', '(', '(', '(']
['(', '[', '(', '{', '(', '{', '[', '{', '(', '(', '<', '(', '[']
['<', '(', '(', '[', '<', '<', '{', '{', '<', '(', '<', '<', '<', '[']
['<', '<', '[', '[', '{', '{', '(', '[']
['<', '<', '(', '{', '(', '{', '{', '(', '[', '(', '<', '{', '<', '{', '[']
['{', '[', '{', '[', '[', '<', '[', '(', '(', '<', '(', '(', '[', '{']
['(', '<', '[', '(', '[', '{', '(', '{', '<', '(', '<', '[', '(', '[', '{']
['{', '(', '[', '(', '[', '(', '(', '{', '[', '(', '[', '(', '(', '[']
['{', '(', '[', '[', '{', '[', '<', '<', '[', '[', '(', '(', '[', '[', '<']
['(', '{', '[', '<', '{', '{', '[', '<', '{', '<', '<', '(', '<', '{', '(']
['(', '<', '{', '(', '<', '[', '[', '<', '{', '[', '(', '[', '(']
['<', '[', '<', '<', '(', '[', '{', '[', '{', '{', '<']
['{', '(', '{', '[', '<', '{', '(', '<', '<', '(', 

In [144]:
arr.sort()

In [145]:
arr[len(arr)//2]

2391385187

In [146]:
arr

[183449,
 5446096,
 8825864,
 13270968,
 34366812,
 41916842,
 46304364,
 46779223,
 76087331,
 116349723,
 126712098,
 133339362,
 356874314,
 356993346,
 466535939,
 578791661,
 668505572,
 941271409,
 1119668443,
 1159121681,
 1210664836,
 1551554948,
 1698145961,
 1922949608,
 2391385187,
 2756895183,
 2816133597,
 2858155937,
 3650916573,
 3656155909,
 4217310963,
 4230962036,
 4554530297,
 4580044622,
 5198005824,
 5251817999,
 5311638568,
 5511668458,
 5512771472,
 5736122716,
 10839418066,
 17034119799,
 17506461621,
 17556164724,
 21136588946,
 22645199869,
 23059473694,
 27175232813,
 27407414683]